# Prepare scrapping en.antaranew.com

In [283]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import math
from datetime import date
from textblob import TextBlob

In [284]:
chrome_options = Options()
chrome_options.add_argument("--headless")

In [285]:
PATH = 'C:/Users/LENOVO-PC/Documents/New/chrome_driver/chromedriver.exe'
driver = webdriver.Chrome(PATH, options = chrome_options)

# Add keyword and generate the link

In [286]:
keyword = input('Input keyword: ')
#keyword = "honda"

Input keyword: mitsubishi


In [287]:
keyword = keyword.replace(" ", "+")
print(keyword)

mitsubishi


In [288]:
def generate_link_home(keyword = keyword):
    if len(keyword.split()) == 2:
        keyword = keyword.replace(" ", "+")
    link = "https://en.antaranews.com/search?q=" + keyword + "&startDate=&endDate=&submit=Submit"
    return link

In [289]:
link_home = generate_link_home()
print(link_home)

https://en.antaranews.com/search?q=mitsubishi&startDate=&endDate=&submit=Submit


# Find the total article found

In [290]:
def total_article():
    link_home = generate_link_home()
    driver.get(link_home)
    element = driver.find_element_by_xpath('//*[@id="main-container"]/div[2]/div/div[1]/div[1]/div[1]/h1/span')
    href = element.text
    href = href.split()
    num = int(href[3])
    return num

In [291]:
%%time
num_of_article = total_article()
print("Found", num_of_article, "of", keyword + "'s article")

Found 20 of mitsubishi's article
Wall time: 2.47 s


# Decide number of pages will scrap

In [292]:
def estimate_pages(num, article_num):
    pages = math.ceil(article_num/num) 
    return pages

In [293]:
article_per_page = 10

In [294]:
num_pages = estimate_pages(article_per_page, num_of_article)
print("Number of page is going to scrap", num_pages)

Number of page is going to scrap 2


# Scrap link in one page

In [295]:
def open_page(page):
    driver.get(page)

In [296]:
def scrap_one_link(article_num, link_store):
    element = driver.find_element_by_xpath('//*[@id="main-container"]/div[2]/div/div[1]/div[2]/div/article[' + str(article_num) + ']/header/h3/a')
    href = element.get_attribute('href')
    link_store.append(href)

In [297]:
def scrap_link_page(link_page, article_num, store):
    open_page(link_page)
    for i in range(article_num):
        i += 1
        scrap_one_link(i, store)

In [298]:
link_ = []
#scrap_link_page(link_home, article_per_page, link_)
link_

[]

## Generate First Page link

In [299]:
def generate_first_page(link_pages, keyword_ = keyword):
    link = "https://en.antaranews.com/search?q=" + keyword_ + "&startDate=&endDate=&submit=Submit"
    link_pages.append(link)
    return link_pages

In [300]:
link_page = []
generate_first_page(link_page)
print(link_page)

['https://en.antaranews.com/search?q=mitsubishi&startDate=&endDate=&submit=Submit']


## Generate The Rest Page Link

In [301]:
def generate_rest_page(link_pages, page_num = num_pages, keyword_ = keyword):
    if page_num > 0:
        for i in range(2,page_num+1):
            link = "https://en.antaranews.com/search/" + keyword_ + "/" + str(i)
            link_pages.append(link)
    return link_pages

In [302]:
#link_page = []
generate_rest_page(link_page)
link_page

['https://en.antaranews.com/search?q=mitsubishi&startDate=&endDate=&submit=Submit',
 'https://en.antaranews.com/search/mitsubishi/2']

# Scrap Link in Last Page

## Counting the rest of article

In [303]:
#Count the rest article
def count_rest(n = num_of_article, a = article_per_page):
    cnt = n%a
    if cnt == 0:
        cnt = 10
        return cnt
    else:
        return cnt

In [304]:
rest = count_rest()
rest

10

## Scrap Last Page

In [305]:
def scrap_last_page(link_page, num_rest, store):
    open_page(link_page)
    for i in range(num_rest):
        i += 1
        scrap_one_link(i, store)

In [306]:
link_ = []
last_page = link_page[num_pages-1]
scrap_last_page(last_page, rest, link_)
link_

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-container"]/div[2]/div/div[1]/div[2]/div/article[6]/header/h3/a"}
  (Session info: headless chrome=96.0.4664.110)


# Scrap All Article Found (Link)

In [307]:
%%time
article_link = []
for i in range(len(link_page)):
    link = link_page[i]
    print(i+1, "/", len(link_page))
    if i != len(link_page)-1 :
        scrap_link_page(link, article_per_page, article_link)
    else :
        scrap_last_page(link, rest, article_link)

1 / 2
2 / 2


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main-container"]/div[2]/div/div[1]/div[2]/div/article[6]/header/h3/a"}
  (Session info: headless chrome=96.0.4664.110)


In [308]:
len(article_link)

15

In [309]:
article_link

['https://en.antaranews.com/news/175838/mitsubishi-power-aero-delivers-vital-power-project-to-mexico-rfp-issuance-to-commercial-operation-in-145-days',
 'https://en.antaranews.com/news/171750/pw-power-systems-will-now-be-mitsubishi-power-aero-signifying-greater-integration-of-the-companies',
 'https://en.antaranews.com/news/162884/mitsubishi-power-receives-order-for-facility-upgrade-at-the-makban-geothermal-power-plant-in-the-philippines',
 'https://en.antaranews.com/news/155486/mitsubishi-power-established-with-renewed-commitment-to-transforming-energy-systems-around-the-world',
 'https://en.antaranews.com/news/146836/mhps-announces-new-company-name-mitsubishi-power',
 'https://en.antaranews.com/news/145294/mhps-world%E2%80%99s-most-efficient-power-plant-is-synced-to-the-grid-and-operating-at-full-load-ahead-of-schedule',
 'https://en.antaranews.com/news/140076/bkpm-mitsubishi-chemical-deliberate-on-investment-expansion-plan',
 'https://en.antaranews.com/news/128557/mitsubishi-motors-

# Scrap Article Details

## Get Article Title

In [310]:
def open_article_link(link):
    driver.get(link)

In [311]:
def get_article_title():
    element = driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/div/div[1]/article/header/h1')
    title = element.text
    return title

In [312]:
open_article_link(article_link[0])
print(get_article_title())

Mitsubishi Power Aero delivers vital power project to Mexico: RFP issuance to commercial operation in 145 days


## Get All Article Title

In [313]:
for i in article_link:
    open_article_link(i)
    #print(get_article_title())

## Get Article Release Date

In [314]:
def get_date():
    element = driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/div/div[1]/article/header/p/span')
    return element.text

In [315]:
open_article_link(article_link[1])
print(get_date())

6th April 2021


## Change Recent Release Article to This Date

In [316]:
def recent_date():
    recent = date.today()
    return str(recent)

In [317]:
recent_date()

'2021-12-28'

# Scrap Title and Date

In [318]:
article_ = []
date_ = []

In [319]:
def scrap_(link_list):
    article = []
    date = []
    n = 1
    for i in link_list:
        open_article_link(i)
        article.append(get_article_title())
        #print(get_article_title())
        j = get_date()
        num = j.split()
        if num[2] == "ago":
            j = recent_date()
            date.append(j)
        else :
            date.append(j)
        print(str(n)+"/"+str(len(link_list)))
        n += 1
    return article, date

In [320]:
%%time
article_,date_ = scrap_(article_link)

1/15
2/15
3/15
4/15
5/15
6/15
7/15
8/15
9/15
10/15
11/15
12/15
13/15
14/15
15/15
Wall time: 19.3 s


In [321]:
df = pd.DataFrame({'date': date_,
                   'title': article_,
                   'link': article_link})

In [322]:
df.head()

,date,title,link
0,4th June 2021,Mitsubishi Power Aero delivers vital power pro...,https://en.antaranews.com/news/175838/mitsubis...
1,6th April 2021,PW Power Systems will now be Mitsubishi Power ...,https://en.antaranews.com/news/171750/pw-power...
2,2nd December 2020,Mitsubishi Power receives order for facility u...,https://en.antaranews.com/news/162884/mitsubis...
3,1st September 2020,Mitsubishi Power established with renewed comm...,https://en.antaranews.com/news/155486/mitsubis...
4,26th April 2020,"MHPS announces new company name ""Mitsubishi Po...",https://en.antaranews.com/news/146836/mhps-ann...


# Date-ing

In [323]:
tmp = []

In [324]:
def date_detection(string):
    string = string.split()
    string = string[0]
    string = int(string[:-2])
    return string

In [325]:
def month_detection(string):
    string = string.split()
    string = string[1]
    month = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    string = month.index(string) + 1
    return string

In [326]:
def year_detection(string):
    string = string.split()
    string = string[2]
    string = int(string)
    return string

In [327]:
print(df.date[1])
print(date_detection(df.date[1]))
print(month_detection(df.date[1]))
print(year_detection(df.date[1]))

6th April 2021
6
4
2021


In [328]:
def date_(string):
    if len(string) > 10:
        year = year_detection(string)
        month = month_detection(string)
        day = date_detection(string)
        d = date(year, month, day)  
        return d.isoformat()
    else:
        return string

In [329]:
date_(df.date[1])

'2021-04-06'

In [330]:
for i in df['date']:
    j = date_(i)
    tmp.append(j)
    print(j)

2021-06-04
2021-04-06
2020-12-02
2020-09-01
2020-04-26
2020-04-02
2020-01-22
2019-07-08
2019-05-18
2018-09-08
2013-01-18
2012-12-14
2012-07-10
2011-07-08
2011-03-03


In [331]:
df.date = tmp
df.head()

,date,title,link
0,2021-06-04,Mitsubishi Power Aero delivers vital power pro...,https://en.antaranews.com/news/175838/mitsubis...
1,2021-04-06,PW Power Systems will now be Mitsubishi Power ...,https://en.antaranews.com/news/171750/pw-power...
2,2020-12-02,Mitsubishi Power receives order for facility u...,https://en.antaranews.com/news/162884/mitsubis...
3,2020-09-01,Mitsubishi Power established with renewed comm...,https://en.antaranews.com/news/155486/mitsubis...
4,2020-04-26,"MHPS announces new company name ""Mitsubishi Po...",https://en.antaranews.com/news/146836/mhps-ann...


In [332]:
file = "result_"+keyword+".csv"
print(file)
df.to_csv(file, index=False)

result_mitsubishi.csv


# Sentiment Analysis TextBloB

In [333]:
df = pd.read_csv("result_"+keyword+".csv")
df.head()

,date,title,link
0,2021-06-04,Mitsubishi Power Aero delivers vital power pro...,https://en.antaranews.com/news/175838/mitsubis...
1,2021-04-06,PW Power Systems will now be Mitsubishi Power ...,https://en.antaranews.com/news/171750/pw-power...
2,2020-12-02,Mitsubishi Power receives order for facility u...,https://en.antaranews.com/news/162884/mitsubis...
3,2020-09-01,Mitsubishi Power established with renewed comm...,https://en.antaranews.com/news/155486/mitsubis...
4,2020-04-26,"MHPS announces new company name ""Mitsubishi Po...",https://en.antaranews.com/news/146836/mhps-ann...


In [334]:
for i in df.title:
    print(i)
    test = TextBlob(i)
    print(test.sentiment)

Mitsubishi Power Aero delivers vital power project to Mexico: RFP issuance to commercial operation in 145 days
Sentiment(polarity=0.05, subjectivity=0.2)
PW Power Systems will now be Mitsubishi Power Aero, signifying greater integration of the companies
Sentiment(polarity=0.5, subjectivity=0.5)
Mitsubishi Power receives order for facility upgrade at the Makban Geothermal Power Plant in the Philippines
Sentiment(polarity=0.0, subjectivity=0.0)
Mitsubishi Power established with renewed commitment to transforming energy systems around the world
Sentiment(polarity=0.0, subjectivity=0.0)
MHPS announces new company name "Mitsubishi Power"
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453)
MHPS: World’s most efficient power plant is synced to the grid and operating at full load, ahead of schedule
Sentiment(polarity=0.425, subjectivity=0.525)
BKPM, Mitsubishi Chemical deliberate on investment expansion plan
Sentiment(polarity=0.0, subjectivity=0.0)
Mitsubishi Motors and 

In [335]:
df.drop_duplicates(subset = 'title', keep = 'first', inplace = True)
df['title'] = df['title'].astype('str')
def get_polarity(text):
    return TextBlob(text).sentiment.polarity
df['Polarity'] = df['title'].apply(get_polarity)

In [336]:
df

,date,title,link,Polarity
0,2021-06-04,Mitsubishi Power Aero delivers vital power pro...,https://en.antaranews.com/news/175838/mitsubis...,0.050000
1,2021-04-06,PW Power Systems will now be Mitsubishi Power ...,https://en.antaranews.com/news/171750/pw-power...,0.500000
2,2020-12-02,Mitsubishi Power receives order for facility u...,https://en.antaranews.com/news/162884/mitsubis...,0.000000
3,2020-09-01,Mitsubishi Power established with renewed comm...,https://en.antaranews.com/news/155486/mitsubis...,0.000000
4,2020-04-26,"MHPS announces new company name ""Mitsubishi Po...",https://en.antaranews.com/news/146836/mhps-ann...,0.136364
5,2020-04-02,MHPS: World’s most efficient power plant is sy...,https://en.antaranews.com/news/145294/mhps-wor...,0.425000
6,2020-01-22,"BKPM, Mitsubishi Chemical deliberate on invest...",https://en.antaranews.com/news/140076/bkpm-mit...,0.000000
7,2019-07-08,Mitsubishi Motors and Mitsubishi Corporation I...,https://en.antaranews.com/news/128557/mitsubis...,0.000000
8,2019-05-18,Indonesia to be Mitsubishi's increasingly sign...,https://en.antaranews.com/news/125698/indonesi...,-0.212500
9,2018-09-08,Mitsubishi jaya elevator to increase local con...,https://en.antaranews.com/news/118310/mitsubis...,0.000000


In [337]:
df['Sentiment_Type'] = ''
df.loc[df.Polarity > 0,'Sentiment_Type']='POSITIVE'
df.loc[df.Polarity == 0,'Sentiment_Type']='NEUTRAL'
df.loc[df.Polarity < 0,'Sentiment_Type']='NEGATIVE'

In [338]:
df.to_csv("result-antara-polarity-" + keyword + ".csv", index=False)

In [339]:
%%time
for i in article_link:
    open_article_link(i)
    article_.append(get_article_title())
    print(get_article_title())
    j = get_date()
    j = j.split()
    if j[2] == "ago":
        j = recent_date()
        date_.append(j)
    else :
        date_.append(j)

Mitsubishi Power Aero delivers vital power project to Mexico: RFP issuance to commercial operation in 145 days


AttributeError: 'function' object has no attribute 'append'

In [340]:
#first page
#2 to last-1 page
#last page

In [341]:
def open_page(first_page):
    first_page = generate_link_home()
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    PATH = 'chromedriver.exe'
    driver1 = webdriver.Chrome(PATH, options = chrome_options)
    driver1.get(first_page)

In [342]:
#scrap link in one page
def scrap_link_1page(link_store,n = article_per_page):
    print(n)
    for j in range(n):
        j += 1
        element = driver.find_element_by_xpath('//*[@id="main-container"]/div[2]/div/div[1]/div[2]/div/article[' + str(j) + ']/header/h3/a')
        href = element.get_attribute('href')
        link_store.append(href)

In [343]:
def close_page():
    driver1.quit()

In [344]:
def scrap_article(n):
    store = []
    for i in n:
        open_page()
        scrap_link_1page(store)
        close_page()
        
    return store

In [345]:
array = scrap_article()

TypeError: scrap_article() missing 1 required positional argument: 'n'

In [ ]:
array

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--headless")
first_page = generate_link_home()
print(first_page)
PATH = 'chromedriver.exe'
driver1 = webdriver.Chrome(PATH, options = chrome_options)
driver1.get(first_page)

link_store = []

n = 10

for j in range(n):
    j += 1
    element = driver.find_element_by_xpath('//*[@id="main-container"]/div[2]/div/div[1]/div[2]/div/article[' + str(j) + ']/header/h3/a')
    href = element.get_attribute('href')
    link_store.append(href)

driver1.quit()

In [ ]:
num = 9
def scrap_link_search(link):
    for i in range(num):
        j = i+1
        element = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/article[' + str(j) + ']/a')
        #element = driver.find_element_by_xpath('/html/body/div[2]/div/div[1]/div[1]/article[' + str(j) + ']/a')
        href = element.get_attribute('href')
        link.append(href)